<a href="https://colab.research.google.com/github/KARENCMP82/Python/blob/main/LLuvia_MIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 5.4 MB/s eta 0:00:00


In [2]:
pip install shap

In [3]:
# gestión de datos
import numpy as np
import pandas as pd

# # fechas
# import datetime as dt
# from datetime import datetime
# from dateutil.relativedelta import relativedelta

# gráficos
import seaborn as sns
import matplotlib.pyplot as plt

# preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

# modelos
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier: el equivalente del Adaline
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression

# metricas
import sklearn.metrics as metrics

# hiperparametrizado
from sklearn.model_selection import GridSearchCV

# interpretabilidad
import shap

# para visualizar mas columnas en un df
pd.set_option('display.max_columns',None)

import warnings
warnings.filterwarnings("ignore")  # inhabilita warnings

np.random.seed(42)

In [5]:
#Conexion
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')
#leemos el fichero IMDB
lluvia=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Ejercicios/lluvia.csv")
# Cargar archivo CSV como DataFrame ver los 5 primero registros
lluvia.head(20)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,Unnamed: 0,fecha,temperatura_max,temperatura_med,temperatura_min,rocio_max,rocio_med,rocio_min,humedad_max,humedad_med,humedad_min,presion_max,presion_med,presion_min,visibilidad_max,visibilidad_med,visibilidad_min,viento_max,viento_med,racha_viento_max,precipitacion,nubes,viento_dir,ind_lluvia,ind_nieve,ind_truenos,ind_niebla,ind_granizo
0,0,1997-01-01,7.0,4.0,2.0,5.0,3.0,2.0,100.0,95.0,76.0,1010,1008,1004,10.0,9.0,4.0,13,6,NaN,0.0,6.0,SW,0,0,0,0,0
1,1,1997-01-02,7.0,3.0,0.0,6.0,3.0,0.0,100.0,92.0,71.0,1007,1003,997,10.0,9.0,4.0,26,8,47.0,0.0,5.0,SE,1,0,0,0,0
2,2,1997-01-03,5.0,3.0,2.0,5.0,1.0,-1.0,100.0,85.0,70.0,1005,999,996,10.0,10.0,7.0,27,19,NaN,0.0,6.0,W,1,1,0,0,0
3,3,1997-01-04,7.0,3.0,-1.0,-2.0,-3.0,-4.0,86.0,63.0,49.0,1012,1010,1005,10.0,10.0,10.0,27,19,40.0,0.0,2.0,W,0,0,0,0,0
4,4,1997-01-05,2.0,0.0,-1.0,2.0,0.0,-3.0,100.0,95.0,86.0,1012,1008,1005,10.0,5.0,1.0,14,6,NaN,0.0,7.0,N,0,1,0,0,0
5,5,1997-01-06,7.0,3.0,1.0,2.0,-1.0,-3.0,100.0,82.0,57.0,1014,1010,1008,10.0,10.0,10.0,11,5,NaN,0.0,4.0,NE,0,0,0,0,0
6,6,1997-01-07,2.0,0.0,-2.0,1.0,-1.0,-3.0,100.0,93.0,75.0,1016,1014,1009,10.0,7.0,0.0,6,2,NaN,0.0,7.0,NE,0,1,0,0,0
7,7,1997-01-08,8.0,4.0,1.0,7.0,4.0,1.0,100.0,96.0,87.0,1015,1005,1003,10.0,8.0,4.0,26,8,NaN,0.0,7.0,W,1,0,0,0,0
8,8,1997-01-09,12.0,10.0,8.0,8.0,3.0,0.0,100.0,65.0,44.0,1015,1008,1003,10.0,10.0,10.0,48,23,48.0,0.0,4.0,W,1,0,0,0,0
9,9,1997-01-10,13.0,8.0,3.0,8.0,5.0,1.0,93.0,83.0,63.0,1021,1018,1016,10.0,10.0,10.0,13,5,NaN,0.0,5.0,E,0,0,0,0,0


Utiliza df.info() para entender los tipos de datos y valores nulos

In [6]:
lluvia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6812 entries, 0 to 6811
Data columns (total 28 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Unnamed: 0        6812 non-null   int64  
 1   fecha             6812 non-null   object 
 2   temperatura_max   6810 non-null   float64
 3   temperatura_med   6809 non-null   float64
 4   temperatura_min   6810 non-null   float64
 5   rocio_max         6810 non-null   float64
 6   rocio_med         6810 non-null   float64
 7   rocio_min         6810 non-null   float64
 8   humedad_max       6810 non-null   float64
 9   humedad_med       6810 non-null   float64
 10  humedad_min       6810 non-null   float64
 11  presion_max       6812 non-null   int64  
 12  presion_med       6812 non-null   int64  
 13  presion_min       6812 non-null   int64  
 14  visibilidad_max   5872 non-null   float64
 15  visibilidad_med   5872 non-null   float64
 16  visibilidad_min   5872 non-null   float64


In [7]:
lluvia.isnull().sum()
#valores nulos

,0
Unnamed: 0,0
fecha,0
temperatura_max,2
temperatura_med,3
temperatura_min,2
rocio_max,2
rocio_med,2
rocio_min,2
humedad_max,2
humedad_med,2


Transforma la columna 'fecha' a formato datetime con pd.to_datetime().

In [8]:
# Convertir la columna 'fecha' a datetime
lluvia['fecha'] = pd.to_datetime(lluvia['fecha'])

Ordena los datos por fecha con df.sort_values().

In [10]:
lluvia.sort_values('fecha', inplace=True)
lluvia.head(5)
#Ordena los datos por feca

,Unnamed: 0,fecha,temperatura_max,temperatura_med,temperatura_min,rocio_max,rocio_med,rocio_min,humedad_max,humedad_med,humedad_min,presion_max,presion_med,presion_min,visibilidad_max,visibilidad_med,visibilidad_min,viento_max,viento_med,racha_viento_max,precipitacion,nubes,viento_dir,ind_lluvia,ind_nieve,ind_truenos,ind_niebla,ind_granizo
0,0,1997-01-01,7.0,4.0,2.0,5.0,3.0,2.0,100.0,95.0,76.0,1010,1008,1004,10.0,9.0,4.0,13,6,NaN,0.0,6.0,SW,0,0,0,0,0
1,1,1997-01-02,7.0,3.0,0.0,6.0,3.0,0.0,100.0,92.0,71.0,1007,1003,997,10.0,9.0,4.0,26,8,47.0,0.0,5.0,SE,1,0,0,0,0
2,2,1997-01-03,5.0,3.0,2.0,5.0,1.0,-1.0,100.0,85.0,70.0,1005,999,996,10.0,10.0,7.0,27,19,NaN,0.0,6.0,W,1,1,0,0,0
3,3,1997-01-04,7.0,3.0,-1.0,-2.0,-3.0,-4.0,86.0,63.0,49.0,1012,1010,1005,10.0,10.0,10.0,27,19,40.0,0.0,2.0,W,0,0,0,0,0
4,4,1997-01-05,2.0,0.0,-1.0,2.0,0.0,-3.0,100.0,95.0,86.0,1012,1008,1005,10.0,5.0,1.0,14,6,NaN,0.0,7.0,N,0,1,0,0,0


Crea la columna lluvia_D+1 usando .shift(-1) para predecir la lluvia del día siguiente

In [11]:
# Crear una columna con el valor del día siguiente
lluvia['valor_D+1'] = lluvia['fecha'].shift(periods=-1)

Define una variable target con el nombre de la columna a predecir.


In [ ]:
# Definimos la variable target
target = 'lluvia_D+1'

2.-**Limpieza y Preprocesamiento de Datos**

In [12]:
# Copiar el DataFrame
lluviaNew = lluvia.copy()

Rellena los valores nulos de las columnas de visibilidad, racha de viento y nubes con el valor del día anterior (ffill). yo he optado por meter la mediana en racha de viento

In [13]:
# Imputar racha_viento_max con la mediana
lluviaNew['racha_viento_max'] = lluviaNew['racha_viento_max'].fillna(lluviaNew['racha_viento_max'].median())

# Imputar nubes con ffill, ffill significa "forward fill" (relleno hacia adelante)
lluviaNew['nubes'] = lluviaNew['nubes'].fillna(method='ffill')

Elimina filas con valores nulos en la columna temperatura_med con df.dropna().

In [15]:
# Eliminar filas donde alguna de las columnas especificadas tiene valores nulos
lluviaNew = lluviaNew.dropna(subset=[
    'temperatura_max', 'temperatura_med', 'temperatura_min',
    'rocio_max', 'rocio_med', 'rocio_min',
    'humedad_max', 'humedad_med', 'humedad_min'
])

Extrae el día y mes de la columna 'fecha' con df['fecha'].dt.day y df['fecha'].dt.month, y elimina la columna original

In [16]:
# Convertir la columna 'fecha' a formato datetime
lluviaNew['fecha'] = pd.to_datetime(lluviaNew['fecha'])

# Extraer el día de la columna 'fecha' y crear una nueva columna 'dia'
lluviaNew['dia'] = lluviaNew['fecha'].dt.day

# Extraer el mes de la columna 'fecha' y crear una nueva columna 'mes'
lluviaNew['mes'] = lluviaNew['fecha'].dt.month

# Extraer el mes de la columna 'fecha' y crear una nueva columna 'año'
lluviaNew['ano'] = lluviaNew['fecha'].dt.year

Realiza One Hot Encoding en la columna 'viento_dir' para convertirla en variables numéricas. 22 viento_dir 6812 non-null object

In [17]:
# Obtener los tipos de datos únicos en la columna 'viento'
tipos_de_datos = lluviaNew['viento_dir'].unique()

print(tipos_de_datos)

['SW' 'SE' 'W' 'N' 'NE' 'E' 'NW' 'S']


In [19]:
# Aplicar One-Hot Encoding
df_encoded = pd.get_dummies(lluviaNew, columns=['viento_dir'])

print(df_encoded)

      Unnamed: 0      fecha  temperatura_max  temperatura_med  \
0              0 1997-01-01              7.0              4.0   
1              1 1997-01-02              7.0              3.0   
2              2 1997-01-03              5.0              3.0   
3              3 1997-01-04              7.0              3.0   
4              4 1997-01-05              2.0              0.0   
...          ...        ...              ...              ...   
6807        6807 2015-12-27             14.0              7.0   
6808        6808 2015-12-28             15.0              8.0   
6809        6809 2015-12-29             11.0              8.0   
6810        6810 2015-12-30              9.0              8.0   
6811        6811 2015-12-31             12.0             10.0   

      temperatura_min  rocio_max  rocio_med  rocio_min  humedad_max  \
0                 2.0        5.0        3.0        2.0        100.0   
1                 0.0        6.0        3.0        0.0        100.0   
2     